# Retrosheet Baseball Data -- Parse Play by Play Data

**Baseball Notebooks**  
1. Downloaded and unzipped baseball data.
2. Helper functions and their motivation for use.
3. Lahman data was wrangled and persisted.
4. This notebook.

Parse the Retrosheet Play by Play data.

The parses used will be the open source parsers by Dr. T. L. Turocy.  
Parser Description: http://chadwick.sourceforge.net/doc/index.html  
Parser Source: https://sourceforge.net/projects/chadwick/

This notebook is designed to be used with Jupyter Lab and the Table of Contents extension: https://github.com/jupyterlab/jupyterlab-toc

## Repeatable Research
All data processing should be documented so that others can repeat the results.  This includes every step from downloading the data through analysis.

### Create Directories for Data Processing

* ~/data/retrosheet/raw -- event files downloaded and unzipped
* ~/data/retrosheet/parsed -- results of running 2 parsers on the event files
* ~/data/retrosheet/collected -- collect the parsed files into dataframes
* ~/data/retrosheet/wrangled -- wrangle the data for analsyis
* ~/data/retrosheet/src -- optional directory to hold parser source code

In [1]:
import os
import re
import wget
from pathlib import Path
import zipfile

In [2]:
# see Baseball Notebook #2
import helper_functions as bb

In [3]:
# create path objects
home = Path.home()
retrosheet = home.joinpath('data/retrosheet')
p_raw = retrosheet.joinpath('raw')
p_wrangled = retrosheet.joinpath('wrangled')

p_parsed = retrosheet.joinpath('parsed')
p_collected = retrosheet.joinpath('collected')
p_src = retrosheet.joinpath('src')

# create directories (if they don't already exist) from these path objects
p_raw.mkdir(parents=True, exist_ok=True)
p_wrangled.mkdir(parents=True, exist_ok=True)
p_parsed.mkdir(parents=True, exist_ok=True)
p_collected.mkdir(parents=True, exist_ok=True)
p_src.mkdir(parents=True, exist_ok=True)

## Parse Event Data for Stats per Player per Game

The event data is in a format that is very difficult to work with.  There is an open-source project which has parsers for the Retrosheet event data.  This project has 6 parsers.  Each of these parsers is fed event data and produces csv or XML or text output.

The two parsers that are of interest for this study are:
1. cwdaily -- player per game stats
2. cwgame -- game stats

The cwbox parser produces a box score in the form MLB fans are accustomed to seeing (or it can produce XML with appropriate tags).  This appears to have the same information as is produced by cwdaily, however cwdaily formats the data as one line per player per game, which is much easier to work with.

The Retrosheet data parser tools are described at:  
http://chadwick.sourceforge.net/doc/index.html  
  
They are distributed under the GPL:  
https://www.gnu.org/licenses/gpl.html  

Note: as of February 2019, the cwdaily parser, published in July 2018, is not described on the above webpage.

### Build Chadwick Parsers on Linux (or use prebuilt Windows binaries)
This section describes how to download the source, compile and install it.

The compile and install procedure here is the standard procedure for compiling and installing open-source code on Linux.

Go To:  
https://sourceforge.net/projects/chadwick/  
Download the source code for version 0.7.1 or later.

If you do not already have a build environment:
1. sudo apt install gcc
2. sudo apt install build-essential

cd to the source directory:
1. ./configure
2. make
3. make install  # or: sudo make install  

Result
1. The cw command line tools will be installed in /usr/local/bin.  
2. The cw library will be installed in /usr/local/lib.  

To allow the command line tools to find the shared libraries, add the following to your .bashrc and then: source .bashrc  
```export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/usr/local/lib```

Optionally copy cwdaily.c and cwgame.c to the src directory.  These C source code files will be parsed to get data dictionary infromation.  These files, and the parsing of these files, is only useful to understanding the data and is not required for the later baseball analysis notebooks.

### Using Prebuilt Windows Binaries
Go To:  
https://sourceforge.net/projects/chadwick/  
Download the Windows binaries for version 0.7.1 or later.

**Linux Wine**  
Install wine: https://wiki.winehq.org/Ubuntu  
Before first use of wine: run winecfg in a terminal

**Windows**  
You could also run the windows binaries on Windows or a Windows VM.

### Run the cwdaily Parser

In [4]:
# normally os.listdir() is used to list a directory
# here, for demonstating the subprocess module, subprocess will be used
# invoke bash directly with shell=False in subprocess
import subprocess

cmd = 'ls /usr/local/bin/cw*'
args = ['/bin/bash', '-c', cmd]
result = subprocess.run(args, shell=False, text=True, capture_output=True)
result.stdout.splitlines()

['/usr/local/bin/cwbox',
 '/usr/local/bin/cwcomment',
 '/usr/local/bin/cwdaily',
 '/usr/local/bin/cwevent',
 '/usr/local/bin/cwgame',
 '/usr/local/bin/cwsub']

In [5]:
# the optionally downloaded C source code for the two parsers
os.listdir(p_src)

['cwgame.c', 'cwdaily.c']

In [6]:
import os
# check the environment variable for LD_LIBRARY_PATH
os.environ['LD_LIBRARY_PATH']

'/usr/local/lib'

In [7]:
# if you are running windows binaries under Linux, 
# prepend 'wine ' to the cmd string below
def process_cwdaily(year):
    """Parse event data into 52 fields of player stats per game.
    
    There are a total of 117 fields to chose from, the first 52 are selected.
    """
    cmd = f'cwdaily -f 0-51 -n -y {year} {year}*.EV*'
    args = ["/bin/bash", "-c", cmd]
    out = f'../parsed/daily{year}.csv'
    with open(out, "w") as outfile:
        result = subprocess.run(args, stdout=outfile)

In [8]:
# change to raw file directory
os.chdir(p_raw)

In [9]:
# parse each year of event data
for year in range(1955, 2019):
    file = p_parsed.joinpath(f'daily{year}.csv')
    
    # if the output file is not already there
    if not file.is_file():
        process_cwdaily(year)

In [10]:
# collect all the parsed files into a single pandas dataframe
import glob
import pandas as pd
import numpy as np

os.chdir(p_parsed)
dailyfiles = glob.glob('daily*.csv')
dailyfiles.sort()

dfs = []
for file in dailyfiles:
    dfs.append(pd.read_csv(file))
player_game = pd.concat(dfs)

In [11]:
player_game = player_game.reset_index(drop=True)
player_game.columns = player_game.columns.str.lower()
player_game.head(3)

,game_id,game_dt,game_ct,appear_dt,team_id,player_id,b_g,b_pa,b_ab,b_r,...,p_bb,p_ibb,p_so,p_gdp,p_hp,p_sh,p_sf,p_xi,p_wp,p_bk
0,BAL195504120,19550412,0,19550412,BOS,goodb101,1,5,5,1,...,0,0,0,0,0,0,0,0,0,0
1,BAL195504120,19550412,0,19550412,BOS,joose101,1,5,4,0,...,0,0,0,0,0,0,0,0,0,0
2,BAL195504120,19550412,0,19550412,BOS,throf101,1,5,5,1,...,0,0,0,0,0,0,0,0,0,0


### Persist Dataframe

Parsing dates and other data wrangling is performed in the next notebook.  This notebook just downloads, parses, and saves the data to a compressed csv file.

Due to the sparsity of the player_game dataframe, using gzip will reduce the file size by a factor of 10+.

In [12]:
bb.mem_usage(player_game)

'1983.76 MB'

In [13]:
player_game.get_dtype_counts()

int64     49
object     3
dtype: int64

In [14]:
player_game = bb.optimize_df_dtypes(player_game)

In [15]:
bb.mem_usage(player_game)

'224.44 MB'

In [ ]:
player_game.get_dtype_counts()

In [16]:
# change working dir
os.chdir(p_collected)

# persist as compressed csv file
%time bb.to_csv_with_types(player_game, 'player_game.csv.gz')

CPU times: user 2min 40s, sys: 191 ms, total: 2min 41s
Wall time: 2min 41s


## Parse Event Data for Stats per Game
Additional information about each game is available, such as the attendance, temperature at game start time, game start time, etc.

In [17]:
# if you are running windows binaries under Linux, prepend 'wine ' to the cmd string below
def process_cwgame(year):
    """Parse yearly event data into 45 fields of game data per year.
    
    For each game, there are 84 standard fields and 95 extended fields to chose from.  
    Only the first 46 standard fields are chosen.
    """
    cmd = f'cwgame -f 0-45 -n -y {year} {year}*.EV*'
    args = ["/bin/bash", "-c", cmd]
    out = f'../parsed/game{year}.csv'
    with open(out, "w") as outfile:
        result = subprocess.run(args, stdout=outfile)

In [18]:
# change to raw file directory
os.chdir(p_raw)

In [19]:
# parse each year of event data
for year in range(1955, 2019):
    file = p_parsed.joinpath(f'game{year}.csv')
    
    # if the output file is not already there
    if not file.is_file():
        process_cwgame(year)

In [20]:
# collect all the parsed files into a single pandas dataframe
import glob
os.chdir(p_parsed)
gamefiles = glob.glob('game*.csv')
gamefiles.sort()

dfs = []
for file in gamefiles:
    dfs.append(pd.read_csv(file))
game = pd.concat(dfs)

In [21]:
game.reset_index(drop=True)
game.columns = game.columns.str.lower()
game.head(3)

,game_id,game_dt,game_ct,game_dy,start_game_tm,dh_fl,daynight_park_cd,away_team_id,home_team_id,park_id,...,away_hits_ct,home_hits_ct,away_err_ct,home_err_ct,away_lob_ct,home_lob_ct,win_pit_id,lose_pit_id,save_pit_id,gwrbi_bat_id
0,BAL195504120,19550412,0,Tuesday,0,F,D,BOS,BAL,BAL11,...,13,5,0,2,8,9,sullf101,colej101,NaN,NaN
1,BAL195504180,19550418,0,Monday,0,F,N,NYA,BAL,BAL11,...,8,3,0,1,5,4,fordw101,moorr101,NaN,NaN
2,BAL195504220,19550422,0,Friday,0,F,N,WS1,BAL,BAL11,...,4,8,2,1,6,11,mcdem102,wilsj104,schmj101,NaN


In [22]:
bb.mem_usage(game)

'185.21 MB'

In [23]:
game.get_dtype_counts()

float64     1
int64      22
object     23
dtype: int64

In [24]:
game = bb.optimize_df_dtypes(game)

In [25]:
bb.mem_usage(game)

'29.36 MB'

In [26]:
game.get_dtype_counts()

category    21
float64      1
int64        3
object       2
uint16       2
uint32       1
uint8       16
dtype: int64

In [28]:
os.chdir(p_collected)
%time bb.to_csv_with_types(game, 'game.csv.gz')

CPU times: user 4.46 s, sys: 16 ms, total: 4.48 s
Wall time: 4.48 s
